In [16]:
import pandas as pd
import streamlit as st
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import altair as alt

In [17]:
comments_df = pd.read_csv('./dataset/comments_sentiment.csv')

In [18]:
comments_df = comments_df.dropna(subset=['Comments'])

In [19]:
# Step 1: Preprocessing text (Vectorization menggunakan TF-IDF)
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_df=0.7)  # Mengabaikan kata yang terlalu sering muncul
tfidf_matrix = tfidf_vectorizer.fit_transform(comments_df['Comments'])  # Representasi teks dalam bentuk vektor

In [20]:
# Step 2: Melakukan Clustering dengan K-Means
num_clusters = 3  # Tentukan jumlah cluster yang diinginkan
kmeans = KMeans(n_clusters=num_clusters, random_state=42)
kmeans.fit(tfidf_matrix)

KMeans(n_clusters=3, random_state=42)

In [21]:
# Step 3: Menambahkan label cluster ke DataFrame
comments_df['Cluster'] = kmeans.labels_

In [22]:
# Step 4: Visualisasi hasil clustering
def plot_clusters(data, labels, title="K-Means Clustering"):
    plt.figure(figsize=(10, 6))
    plt.scatter(data[:, 0], data[:, 1], c=labels, cmap='rainbow')
    plt.title(title)
    plt.show()

In [23]:
# Mengurangi dimensionalitas ke 2D untuk visualisasi (menggunakan PCA)
from sklearn.decomposition import PCA

pca = PCA(n_components=2)
reduced_tfidf = pca.fit_transform(tfidf_matrix.toarray())

In [24]:
# Menampilkan hasil clustering dalam bentuk scatter plot
plot_clusters(reduced_tfidf, kmeans.labels_, title="K-Means Clustering with 3 Clusters")

C:\Users\Naoval\AppData\Local\Temp\ipykernel_6548\2103636269.py:6: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [25]:
# Mengelompokkan data berdasarkan cluster
clustered_data = comments_df.groupby('Cluster')

# Menampilkan data dari masing-masing cluster
for cluster, group in clustered_data:
    print(f"Data untuk Cluster {cluster}:")
    print(group[['Comments', 'Cluster']])
    print("\n")

Data untuk Cluster 0:
                                               Comments  Cluster
6     wohahahahaha brand hp paling bnyak rusak di ko...        0
20    yang di bilang apa di jawab apa kebanyakan cheers        0
27    yang di kritik apa yang dijawab apa. pemahaman...        0
28    di kritik gak makin banyak perubahan malah mak...        0
38    jual barang di indo bahasanya campur&quot; ban...        0
...                                                 ...      ...
2149  waktu kritik pertama haptic alesanya lebih fok...        0
2151                  nahh      gw jg bertanya² di situ        0
2158  nah iya, dari zaman liat review di kaskus biki...        0
2159  sebetulnya ya... produknya banyak yang bagus. ...        0
2167  di reno12 series, kita bisa rasain futur ai ya...        0

[273 rows x 2 columns]


Data untuk Cluster 1:
                                               Comments  Cluster
2                            kelihatan kualitasnya wkwk        1
3                   

In [26]:
from wordcloud import WordCloud
import matplotlib.pyplot as plt

# Fungsi untuk membuat WordCloud dari cluster
def generate_wordcloud(data, cluster_label):
    # Menggabungkan semua teks di cluster yang sama
    text = " ".join(review for review in data)
    
    # Membuat WordCloud
    wordcloud = WordCloud(width=800, height=400, background_color="white").generate(text)
    
    # Menampilkan WordCloud
    plt.figure(figsize=(10, 5))
    plt.imshow(wordcloud, interpolation="bilinear")
    plt.axis("off")
    plt.title(f"WordCloud for Cluster {cluster_label}")
    plt.show()

# Mendapatkan WordCloud untuk setiap cluster
for cluster in comments_df['Cluster'].unique():
    cluster_data = comments_df[comments_df['Cluster'] == cluster]['Comments']
    generate_wordcloud(cluster_data, cluster)


C:\Users\Naoval\AppData\Local\Temp\ipykernel_6548\2925482188.py:17: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [31]:
from collections import Counter
import re

def get_top_words(data, n=10):
    # Menggabungkan semua teks dalam satu cluster
    text = " ".join(data)
    
    # Menghapus karakter non-alfabet (hanya menyimpan kata)
    words = re.findall(r'\b\w+\b', text.lower())
    
    # Hitung frekuensi kata
    word_counts = Counter(words)
    
    # Ambil n kata teratas
    top_words = word_counts.most_common(n)
    return top_words

# Dapatkan kata-kata teratas untuk setiap cluster
top_words_per_cluster = {}
for cluster in comments_df['Cluster'].unique():
    cluster_data = comments_df[comments_df['Cluster'] == cluster]['Comments']
    top_words_per_cluster[cluster] = get_top_words(cluster_data, n=10)


In [32]:
import pandas as pd
import altair as alt

# Persiapkan data untuk visualisasi
top_words_df = []

for cluster, words in top_words_per_cluster.items():
    for word, count in words:
        top_words_df.append({'Cluster': cluster, 'Word': word, 'Count': count})

top_words_df = pd.DataFrame(top_words_df)

# Buat diagram batang dengan Altair
bar_chart = alt.Chart(top_words_df).mark_bar().encode(
    x=alt.X('Count:Q', title='Frequency'),
    y=alt.Y('Word:N', title='Words', sort='-x'),
    color='Cluster:N',
    column='Cluster:N'  # Memisahkan diagram batang berdasarkan cluster
).properties(
    title='Most Common Words per Cluster'
)

# Tampilkan diagram batang di Streamlit
st.altair_chart(bar_chart, use_container_width=True)


2024-10-14 11:53:52.711 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:53:52.712 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [28]:
# Step 5: Visualisasi dengan Altair
st.title("Clustering Hasil Analisis Sentimen")
st.subheader("Visualisasi Clustering dengan K-Means")

2024-10-14 11:43:44.748 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:43:44.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:43:44.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:43:44.749 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [29]:
# Membuat visualisasi menggunakan Altair
chart = alt.Chart(comments_df).mark_circle(size=60).encode(
    x='Cluster:N',
    y='like:Q',
    color='Cluster:N',
    tooltip=['Comments', 'Cluster']
).interactive()

st.altair_chart(chart, use_container_width=True)

2024-10-14 11:43:44.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:43:44.777 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [30]:
# Menampilkan hasil clustering dalam tabel
st.write(comments_df[['Comments', 'Cluster']].head(10))

2024-10-14 11:43:44.785 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-10-14 11:43:44.787 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
